In [1]:
from datetime import datetime, date
# Imports for 'inspect/explore/reflect' columns
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

engine = create_engine("sqlite:///hawaii.sqlite")

In [2]:
# Import dependencies for 
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#### Using 'inspect' to look at table columns and types. aka 'explore'

In [3]:
inspector = inspect(engine)
inspector.get_table_names()

['measurements', 'stations']

In [4]:
columns = inspector.get_columns('stations')
for column in columns:
    print(column)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': VARCHAR(length=255), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'name', 'type': VARCHAR(length=255), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'latitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'longitude', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'elevation', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


In [5]:
m_columns = inspector.get_columns('measurements')
for column in m_columns:
    print(column)

{'name': 'id', 'type': INTEGER(), 'nullable': False, 'default': None, 'autoincrement': 'auto', 'primary_key': 1}
{'name': 'station', 'type': VARCHAR(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'date', 'type': DATE(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'prcp', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}
{'name': 'tobs', 'type': FLOAT(), 'nullable': True, 'default': None, 'autoincrement': 'auto', 'primary_key': 0}


#### Using 'reflection' to view columns

In [6]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurements', 'stations']

In [7]:
Measurements = Base.classes.measurements
Stations = Base.classes.stations

In [8]:
list(Stations.__table__.columns)

[Column('id', INTEGER(), table=<stations>, primary_key=True, nullable=False),
 Column('station', VARCHAR(length=255), table=<stations>),
 Column('name', VARCHAR(length=255), table=<stations>),
 Column('latitude', FLOAT(), table=<stations>),
 Column('longitude', FLOAT(), table=<stations>),
 Column('elevation', FLOAT(), table=<stations>)]

In [9]:
list(Measurements.__table__.columns)

[Column('id', INTEGER(), table=<measurements>, primary_key=True, nullable=False),
 Column('station', VARCHAR(), table=<measurements>),
 Column('date', DATE(), table=<measurements>),
 Column('prcp', FLOAT(), table=<measurements>),
 Column('tobs', FLOAT(), table=<measurements>)]

#### Creating the query sessions.  First, check the query implementation by calling all the columns from both tables and storing into variables, then used a for-loop to check the data from the query, and to double-check, imported into pandas dataframe to check if datetime was imported correctly.

In [10]:
session = Session(engine)
station_query = (session.query(Stations.id, Stations.station, Stations.name,
                     Stations.latitude, Stations.longitude, Stations.elevation)
         .all())
# for row in station_query:
#     print(row)

In [11]:
measurement_query = (session.query(Measurements.id, Measurements.station,
                                  Measurements.date, Measurements.prcp,
                                  Measurements.tobs).all())
#  **Storing 'Measurements' table -
#  measurement_query_2 = session.query(Measurements)
# for row in measurement_query:
#      print(row.date.month)

In [12]:
station_df = pd.DataFrame(station_query)
measurements_df = pd.DataFrame(measurement_query)

In [13]:
#station_df.head()

In [14]:
#measurements_df.head()

#### Now the fun begins

In [30]:
prcp_query = session.query(Measurements.date, Measurements.prcp).filter(Measurements.date >= 
                                                                        date(year=2016, month=8, day=23)).all()
# for row in prcp_query:
#     print(row)

In [64]:
prcp_query_df = pd.DataFrame(prcp_query)
prcp_query_df = prcp_query_df.sort_values('date').set_index('date')
#prcp_query_df.head()

In [65]:
#df = df.drop(df[df.score < 50].index)
prcp_query_df = prcp_query_df.reset_index()
prcp_query_df = prcp_query_df[prcp_query_df.prcp > -1].set_index('date')
#prcp_query_df.head()

In [66]:
prcp_query_sum = prcp_query_df.groupby('date').mean()
prcp_query_sum.head()

,prcp
date,
2016-08-23,0.451667
2016-08-24,1.555000
2016-08-25,0.077143
2016-08-26,0.016667
2016-08-27,0.064000
